# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project6/2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [4]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6/2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [5]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

# 더 필요한 라이브러리 추가 -------------




### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [11]:
emergency_room = pd.read_csv(path + 'emergency_room.csv')
emergency_room

,dutyName,dutyAddr,latitude,longitude
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),37.240316,127.214491
...,...,...,...,...
521,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",37.393404,126.924477
522,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",37.358645,126.947360
523,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),35.185413,129.121459
524,흑룡의원,인천광역시 옹진군 백령면 백령로 831,37.959524,124.665499


## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 임의의 두 좌표간 거리 계산
    * 응급실 데이터프레임을 열어서
    * 응급실 두 곳의 좌표를 확인하고
    * 두 지점의 거리를 계산해 봅시다
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [10]:
from haversine import haversine

# 테스트
my_location = (37.393031447152, 127.22470530199)
emergency = (37.234641, 127.210499)

# 거리 계산
distance = haversine(my_location, emergency, unit='km')
print(f"현재 내 위치와 입력된 응급실 간의 거리: {distance:.2f} km")

현재 내 위치와 입력된 응급실 간의 거리: 17.66 km


#### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후
    * 가장 가까운 거리의 응급실 3 곳을 반환합니다.
* 이를 함수로 생성하고 테스트 해 봅시다.

In [14]:
def find_nearest_emergency_rooms(my_location, emergency_room_df, n=3):
    # 거리 계산 및 데이터프레임에 추가
    emergency_room_df['distance'] = emergency_room_df.apply(
        lambda row: round(haversine(my_location, (row['latitude'], row['longitude']), unit='km'), 2), axis=1
    )

    # 거리 기준으로 정렬하고 가장 가까운 n개 응급실 선택
    nearest_rooms = emergency_room_df.nsmallest(n, 'distance')

    return nearest_rooms[['dutyName', 'dutyAddr', 'latitude', 'longitude', 'distance']]

In [15]:
# 현재 위치
my_location = (37.393031447152, 127.22470530199)

# 가장 가까운 응급실 3곳 찾기
nearest_emergency_rooms = find_nearest_emergency_rooms(my_location, emergency_room)
print(nearest_emergency_rooms)

    dutyName                       dutyAddr   latitude   longitude  distance
443    참조은병원          경기도 광주시 광주대로 45 (경안동)  37.410954  127.260224      3.72
76    국군수도병원  경기도 성남시 분당구 새마을로177번길 81 (율동)  37.391867  127.148586      6.73
142   바른마디병원      경기도 성남시 중원구 광명로 330 (금광동)  37.445270  127.162566      7.99


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율 적입니다.
* 해결 방안 : 그래서 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정 합니다.
* hint :
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
    * 응급실 정보에서 해당 구간을 먼저 조회한 후
    * 거리 계산

In [27]:
def find_nearest_emergency_rooms_update(my_location, emergency_room_df, n=3, max_distance=10):
    # 거리 계산 및 데이터프레임에 추가
    emergency_room_df['distance'] = emergency_room_df.apply(
        lambda row: round(haversine(my_location, (row['latitude'], row['longitude']), unit='km'), 2), axis=1
    )

    # 10km 이내의 응급실만 필터링
    nearby_rooms = emergency_room_df[emergency_room_df['distance'] <= max_distance]

    # 거리 기준으로 정렬하고 가장 가까운 n개 응급실 선택
    nearest_rooms = nearby_rooms.nsmallest(n, 'distance')

    return nearest_rooms[['dutyName', 'dutyAddr', 'latitude', 'longitude', 'distance']]

In [22]:
# 현재 위치
my_location = (37.393031447152, 127.22470530199)

# 가장 가까운 응급실 3곳 찾기
nearest_emergency_rooms = find_nearest_emergency_rooms_update(my_location, emergency_room, 3, 10)
nearest_emergency_rooms

,dutyName,dutyAddr,latitude,longitude,distance
443,참조은병원,경기도 광주시 광주대로 45 (경안동),37.410954,127.260224,3.72
76,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),37.391867,127.148586,6.73
142,바른마디병원,경기도 성남시 중원구 광명로 330 (금광동),37.445270,127.162566,7.99


In [74]:
def find_nearest_emergency_rooms_update2(my_location, emergency_room_df, n=5, max_distance=10):
    my_latitude, my_longitude = my_location

    # 초기 max_distance 설정
    current_distance = max_distance

    while True:
        # 위도와 경도를 기준으로 대략적인 범위 계산
        lat_range = current_distance / 111  # 약 1도 = 111km
        lon_range = current_distance / (111 * abs(np.cos(np.radians(my_latitude))))  # 경도는 위도에 따라 다름

        # 범위 내의 응급실 필터링
        filtered_rooms = emergency_room_df[
            (emergency_room_df['latitude'] >= my_latitude - lat_range) &
            (emergency_room_df['latitude'] <= my_latitude + lat_range) &
            (emergency_room_df['longitude'] >= my_longitude - lon_range) &
            (emergency_room_df['longitude'] <= my_longitude + lon_range)
        ].copy()  # .copy()를 사용하여 복사본을 생성

        # 거리 계산 및 데이터프레임에 추가
        filtered_rooms.loc[:, 'distance'] = filtered_rooms.apply(
            lambda row: round(haversine(my_location, (row['latitude'], row['longitude']), unit='km'), 2), axis=1
        )

        # 거리 기준으로 정렬하고 가장 가까운 n개 응급실 선택
        nearest_rooms = filtered_rooms.nsmallest(n, 'distance')

        # 응급실 수가 n보다 크거나 같으면 종료
        if len(nearest_rooms) >= n:
            return nearest_rooms[['dutyName', 'dutyAddr', 'latitude', 'longitude', 'distance']]

        # 응급실 수가 n보다 작으면 max_distance를 5km 늘림
        current_distance += 5

        # max_distance가 너무 커지지 않도록 제한 (예: 50km)
        if current_distance > 50:
            print("응급실을 찾을 수 없습니다.")
            return nearest_rooms[['dutyName', 'dutyAddr', 'latitude', 'longitude', 'distance']]


In [75]:
# 현재 위치
my_location = (37.393031447152, 127.22470530199)

# 가장 가까운 응급실 3곳 찾기
nearest_emergency_rooms = find_nearest_emergency_rooms_update2(my_location, emergency_room, 5, 10)
nearest_emergency_rooms

,dutyName,dutyAddr,latitude,longitude,distance
443,참조은병원,경기도 광주시 광주대로 45 (경안동),37.410954,127.260224,3.72
76,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),37.391867,127.148586,6.73
142,바른마디병원,경기도 성남시 중원구 광명로 330 (금광동),37.445270,127.162566,7.99
189,성남중앙병원,경기도 성남시 중원구 산성대로476번길 12 (금광동),37.452803,127.162095,8.65
442,차의과학대학교분당차병원,경기도 성남시 분당구 야탑로 59 (야탑동),37.410115,127.125084,9.00


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [ ]:
# 해당 부분은 보안상 생략합니다.

#### 2) 함수 생성

In [80]:
import requests

def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers=headers, params=params)
    return response.json()

* 테스트

In [77]:
start_lat = 37.393031447152
start_lng = 127.22470530199
dest_lat = 37.410954
dest_lng = 127.260224

info=get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key)
print(info)

{'code': 0, 'message': '길찾기를 성공하였습니다.', 'currentDateTime': '2024-11-20T15:00:44', 'route': {'trafast': [{'summary': {'start': {'location': [127.2247043, 37.3930314]}, 'goal': {'location': [127.2602237, 37.4109534], 'dir': 2}, 'distance': 6441, 'duration': 909120, 'departureTime': '2024-11-20T15:00:44', 'bbox': [[127.2246639, 37.3881517], [127.2612118, 37.4130575]], 'tollFare': 0, 'taxiFare': 10300, 'fuelPrice': 751}, 'path': [[127.2246639, 37.3929511], [127.2249874, 37.3928482], [127.225476, 37.3927087], [127.2257214, 37.3926389], [127.2259498, 37.3925737], [127.2261343, 37.3924885], [127.2261501, 37.3924822], [127.2263867, 37.3923565], [127.2265158, 37.3922721], [127.2266245, 37.3921903], [127.2266359, 37.3921832], [127.2267378, 37.3921212], [127.2268612, 37.3920359], [127.2269099, 37.39199], [127.2269485, 37.3919495], [127.2273377, 37.3914844], [127.2275258, 37.3913163], [127.2276821, 37.3912058], [127.2279119, 37.3910819], [127.2283192, 37.390908], [127.2286021, 37.3907878], [127.22

#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [95]:
import pandas as pd
import requests
from haversine import haversine

# 추천 응급실 찾기 함수
def recommend_hospital3(my_location, emergency_room_df, c_id, c_key):
    nearest_room = find_nearest_emergency_rooms_update2(my_location, emergency_room_df, n=5)

    if not nearest_room.empty:
        distances = []  # 거리 정보를 담을 리스트

        for index, row in nearest_room.iterrows():
            # 각 병원에 대해 거리 계산
            emergency_info = get_dist(my_location[0], my_location[1], row['latitude'], row['longitude'], c_id, c_key)
            if emergency_info is not None:
                distance = emergency_info['route']['trafast'][0]['summary']['distance']
                minTimeTaken = emergency_info['route']['trafast'][0]['summary']['duration']/60000
                distances.append((row['dutyName'], row['dutyAddr'], distance / 1000, round(minTimeTaken, 2)))  # 병원 이름, 주소, 거리 (m-> km), (밀리초 - 분)

        # 거리 기준으로 정렬하여 가장 가까운 3개 병원 선택
        closest_hospitals = sorted(distances, key=lambda x: x[2])[:3]

        # print("가장 가까운 3개의 응급실 정보:")
        # for hospital in closest_hospitals:
        #     print(f"병원 이름: {hospital[0]}, 주소: {hospital[1]}, 거리: {hospital[2]} m")

        return closest_hospitals  # 가장 가까운 3개 병원 리스트 반환
    else:
        print("주변에 응급실이 없습니다.")
        return []




In [97]:
# 가장 가까운 응급실 추천
hospitals=recommend_hospital3(my_location, emergency_room, c_id, c_key)
for hospital in hospitals:
    name, address, distance, time = hospital
    print(f"병원 이름: {name}")
    print(f"병원 주소: {address}")
    print(f"이동 거리: {distance} km")
    print(f"소요 시간: {time} 분")
    print()  # 각 병원 정보 사이에 빈 줄 추가

병원 이름: 참조은병원
병원 주소: 경기도 광주시 광주대로 45 (경안동)
이동 거리: 6.317 km
소요 시간: 15.66 분

병원 이름: 바른마디병원
병원 주소: 경기도 성남시 중원구 광명로 330 (금광동)
이동 거리: 10.577 km
소요 시간: 20.31 분

병원 이름: 성남중앙병원
병원 주소: 경기도 성남시 중원구 산성대로476번길 12 (금광동)
이동 거리: 11.285 km
소요 시간: 22.72 분

